# Conto Corrente in python


nella  lezione verrà mostrato come creare una classe che corrisponda ad un conto corrente bancario in cui è possibile eseguire le comuni azioni:
- apertura conto
- deposito
- prelievo
- richiesta fido
- richiesta prestito

la classe `Banca` definirà l'istituo in cui viene aperto il conto Corrente


In [54]:
class Persona:
    
    def __init__(self, nome, cognome):
        self.nome = nome
        self.cognome = cognome
        
    # metodo che ci permette di ritornare il nome completo della persona
    # attraverso l'uninone delle proprietà nome e cognome
    def nome_completo(self):
        return f"{self.nome} {self.cognome}"
    
    # permette di stampare a scherom un determinato messaggio
    # nel formato:
    # Nome Completo> messaggio
    def parla(self, message):
        print(f"{self.nome_completo()}> {message}")
        

In [10]:
class Carta:
    
    def __init__(self, tipologia: str, prelievo_max: float = 0, operazione_max: float = 0):
        self.tipologia = tipologia
        self.prelievo_max = prelievo_max
        self.operazione_max = operazione_max
        
class CartaDebito(Carta):
    def __init__(self, prelievo_max: float = 500.0, operazione_max: float = 1000.0):
        super().__init__('debito', prelievo_max, operazione_max)

class CartaCredito(Carta):
    def __init__(self, prelievo_max: float = 1500.0, operazione_max: float = 1500.0):
        super().__init__('credito', prelievo_max, operazione_max)

In [11]:
c = CartaDebito(1500, 1000)
print(c.operazione_max)
print(c.prelievo_max)

1000
1500


In [56]:
import uuid
import logging
import time

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("ContoCorrente")


class ContoCorrente:
    
    def __init__(self, owner: Persona, saldo: int = 0, aperto: bool = True, fido: bool = False, **kwargs):
        self.owner = owner
        self.saldo = saldo
        self.aperto = aperto
        self.numero_conto = uuid.uuid4().hex
        self._events = []
        self._cards = []
        self.apri()
        if fido:
            self.richiedi_fido()
        else:
            self.fido = 0
            self.fido_max = 0
        for k, v in kwargs.items():
            setattr(self, k, v)

    def genera_evento(self, message, level='info'):
        self._events.append({
            "timestamp" : time.time(),
            "message" : message,
            "level" : level
        })
        
    @property
    def events(self):
        return sorted(self._events, key = lambda i: i['timestamp']) 
    
    @property
    def events_human(self):
        for item in self.events:
            yield f"{self.numero_conto} - {item['timestamp']} -> {item['message']}"
            
    def apri(self):
        logger.info(f"richiesta apertura conto corrente a nome di {self.owner.nome_completo()}")
        self.genera_evento("apertura conto")
        if not self.aperto:
            self.aperto = True
            return
        
    def chiudi(self):
        logger.info("richiesta chiusura conto")
        self.genera_evento("chiusura conto")
        if self.aperto:
            self.aperto = False
        
    def preleva(self, valore: float):
        if self.aperto:
            if (self.saldo ) - valore < (0 - self.fido) :
                raise Exception("importo da prelevare non disponibile")
            
            logger.info(f"richiesta di prelievo: {valore}")
            self.saldo -= valore
            self.genera_evento(f"prelevato {valore}€")
        return
            
    def deposita(self, valore: float):
        if self.aperto:
            logger.info(f"richiesta di deposito {valore}")
            self.saldo += valore
            self.genera_evento(f"depositato {valore}€")
        return
        
    def richiedi_fido(self):
        if getattr(self, 'fido', None):
            logger.error("fido già richiesto in precedenza")
            return
        logger.info("richiesta fido")
        if self.saldo >= 0 and self.saldo <= 1000:
            self.fido = 500
            self.fido_max = self.fido
        elif self.saldo > 1000 and self.saldo< 100000:
            self.fido = 2500
            self.fido_max = self.fido
        else:
            self.fido = 5000
            self.fido_max = self.fido
        logger.info(f"richiesta fido approvata: {self.fido}€")
            
    def saldo_disponibile(self, fido: bool = True) -> float:
        if self.fido:
            return self.saldo + self.fido
        return self.saldo


In [57]:
class Banca:
    
    def __init__(self, nome):
        self.nome = nome
        self.conti_correnti = []
        
    def apri_conto_corrente(self, conto: ContoCorrente) -> ContoCorrente:
        self.conti_correnti.append(conto)
        
    def cerca_conto(self, numero: str):
        for item in self.conti_correnti:
            if item.numero_conto == numero:
                return item
        raise Exception(f"conto {numero} non trovato")
        
    def chiudi_conto_corrente(self, conto: ContoCorrente):
        self.cerca_conto(conto.numero_conto)
        
    
        

In [60]:
banca = Banca("Python")

def apri_conto_uno(banca: Banca):
    persona = Persona(nome="Mario", cognome="Rossi")
    conto = ContoCorrente(owner=persona, saldo = 3000)
    banca.apri_conto_corrente(conto)
    return conto

conto = apri_conto_uno(banca)
mainlogger = logging.getLogger("__main__")
mainlogger.info(f"saldo disponibile: {conto.saldo_disponibile()}")
mainlogger.info(f"fido: {conto.fido}")
mainlogger.info(f"saldo reale: {conto.saldo}")
conto.deposita(10000)
conto.preleva(5000)
print("\n".join(conto.events_human))
mainlogger.info(f"saldo disponibile: {conto.saldo_disponibile()}")
conto.richiedi_fido()
mainlogger.info(f"saldo disponibile: {conto.saldo_disponibile()}")

INFO:ContoCorrente:richiesta apertura conto corrente a nome di Mario Rossi
INFO:__main__:saldo disponibile: 3000
INFO:__main__:fido: 0
INFO:__main__:saldo reale: 3000
INFO:ContoCorrente:richiesta di deposito 10000
INFO:ContoCorrente:richiesta di prelievo: 5000
INFO:__main__:saldo disponibile: 8000
INFO:ContoCorrente:richiesta fido
INFO:ContoCorrente:richiesta fido approvata: 2500€
INFO:__main__:saldo disponibile: 10500


033e14b605fb4c8381fc304478585ea4 - 1580675663.4497747 -> apertura conto
033e14b605fb4c8381fc304478585ea4 - 1580675663.4535398 -> depositato 10000€
033e14b605fb4c8381fc304478585ea4 - 1580675663.454458 -> prelevato 5000€
